In [ ]:
import os
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
OUT_PATH = "outs_final_nee"
os.makedirs(OUT_PATH, exist_ok=True)

# DATA_PATH = "ny/fRNA/fRNA/"


ONE_COL_MM = 890
SPINE_WIDTH = 0.5

nee_fs = 7
font = {
    "family": "sans-serif",
    "sans-serif": ["Helvetica"],
    "weight": "normal",
    "size": nee_fs,
}
plt.matplotlib.rc("font", **font)
# plt.matplotlib.rcParams['text.usetex'] = True

figsize = (ONE_COL_MM / 250, 2.6/4.5 * ONE_COL_MM / 250)

In [ ]:
# Extra path to get results
extra_path = "ny/fRNA/fRNA/"

In [ ]:
def Aborted(x, threshold):
    if x < threshold:
        return 0
    return 1

In [ ]:
def get_stats(df_total, file_name, cols2, aborted_included="yes"):
    """Get single results from stats output file"""
    L = int(file_name[file_name.find(f"stats_L")+len("stats_L"):file_name.find(".txt")])
    df = pd.read_csv(file_name, sep="\t")
    df["L"] = L

    if L >= 50:
        return df
    # Load other data
    pdata = pd.read_csv(f"{extra_path}data/ps_l"+str(L)+".txt", header=None).rename({0:"phenotype"}, axis=1)
    df["Np_fRNA"] = len(pdata)
    
    df = df[["L", "Np_fRNA"] + [col for col in df.columns if col != "L" and col != "Np_fRNA"]]
    df["total_phenotypes"] = df["u_size"] + df["v_size"]

    threshold = np.max(df["total_phenotypes"])
    df["aborted"] = df["total_phenotypes"].apply(lambda x : Aborted(x, threshold))
    df["aborted_included"] = aborted_included
    df["threshold"] = threshold
    df_out = df.copy()
    return df_out

def SS(L):
    """From: http://dx.doi.org/10.1098/rsfs.2015.0053"""
    return 0.13 * 1.76**L

def get_results(files):
    """Get all results and combine into one dataframe"""
    cols2 = ["L","Tests", "Tests(>=one_success)", "Np_fRNA", "P(MIFP+DFS)", "SE(MIFP+DFS)",
             "AbortedMeanOverall", "P(MIFP+DFS)_all", "SE(MIFP+DFS)_all",
             "AbortedMeanOverall_all", "threshold", "aborted_included"]
    df_total = pd.DataFrame([], columns=cols2, index=[])
    dfs = []
    for aborted_included in ["yes"]:
        for file in files:
            df = get_stats(df_total, file, cols2, aborted_included)
            dfs.append(df)
    dfs = pd.concat(dfs).reset_index(drop=True)
    return dfs

In [ ]:
# Load all results across different thresholds
files = sorted(glob.glob(os.path.join(f"{extra_path}180414_r_assignment_parallel_L*_2e?/data_out/stats_L??.txt")))
dfs = get_results(files)

# Add apprroximate number of phenotypes
dfs["Np"] = dfs["L"].apply(SS)
dfs["logNp"] = dfs["Np"].apply(np.log10)

In [ ]:
# Make figure for navigability and aborted fraction against threshold
n = 0
fig, axs = plt.subplots(2, 1, figsize=(4,4), sharex=True, gridspec_kw={'height_ratios': [1, 2]})
upper_lims = {20:2e6, 25:2e6, 30:2e7, 35:2e10, 40:2e10}
ms = 3

for name, group_ in dfs.groupby("L"):
    ax = axs[0]
    ax.text(0.85, 1.1, r"A")
    group_psi = group_[group_["aborted"]!=1].groupby(["threshold"])["Fittest_found?"].mean().to_frame().reset_index().rename(columns={"Fittest_found?":"nav"})
    group = group_.groupby(["threshold", "L"])["aborted"].mean().round(3).to_frame().reset_index().rename(columns={"aborted":"AbortedMeanOverall"})
    group_L = int(group.L.iloc[0])
    ax.plot(np.log10(group_psi.threshold), group_psi["nav"], label="L={0}".format(group.L.iloc[0]), linestyle="-", marker="o", color="C"+str(n), ms=ms)
    
    ax = axs[1]
    ax.text(0.85, 3, r"B")
    ax.plot(np.log10(group.threshold), group["AbortedMeanOverall"], label="L={0}".format(group.L.iloc[0]), linestyle="-", marker="o", color="C"+str(n), ms=ms)

    x = np.log10(group.threshold)
    y = group["AbortedMeanOverall"]

    z = np.polyfit(x, y, 2)
    f = np.poly1d(z)
    
    x_new = np.linspace(np.log10(2e3), np.log10(upper_lims[group_L]), num=100, endpoint=True)
    ax.plot(x_new, f(x_new), label="_nolegend_", linestyle="--", marker="None", color="C"+str(n), alpha=0.5)
    n+=1
    
axs[0].set_xlim(2.5,np.log10(2e10))
axs[0].set_ylim(0,1.05)
axs[0].set_ylabel(r"Navigability, $\left<\psi\right>$")

axs[1].set_xlabel(r"Computational threshold, $\log_{10} T$")
axs[1].set_ylabel(r"Proportion aborted, $\alpha$")
axs[1].set_xlim(2.5,np.log10(2e10))
axs[1].set_ylim(0.01,3)
axs[1].set_yscale('log')

plt.plot([np.log10(3e6), np.log10(3e6)], [-1,2], color="k", zorder=-1, linestyle=":", label="1 month limit", alpha=0.2)
axs[0].legend(loc="lower right", ncol=3, prop={'size':"x-small"})
axs[1].legend(loc="upper right", ncol=3, prop={'size':"x-small"})
plt.savefig(os.path.join(OUT_PATH, "fig5.pdf"), transparent=True, bbox_inches='tight')

In [ ]:
# Make figure for navigability and aborted fraction against threshold
n = 0
# fig, axs = plt.subplots(1, 1, figsize=(4, 2.5), sharex=True)#, gridspec_kw={'height_ratios': [1]})
fig, axs = plt.subplots(1, 1, figsize=(ONE_COL_MM / 250, ONE_COL_MM / 250 * 2.5/4), sharex=True)#, gridspec_kw={'height_ratios': [1]})
upper_lims = {20:2e6, 25:2e6, 30:2e7, 35:2e10, 40:2e10}
ms = 3

for name, group_ in dfs.groupby("L"):
    # ax = axs[0]
    # ax.text(0.85, 1.1, r"A")
    group_psi = group_[group_["aborted"]!=1].groupby(["threshold"])["Fittest_found?"].mean().to_frame().reset_index().rename(columns={"Fittest_found?":"nav"})
    group = group_.groupby(["threshold", "L"])["aborted"].mean().round(3).to_frame().reset_index().rename(columns={"aborted":"AbortedMeanOverall"})
    group_L = int(group.L.iloc[0])
    # ax.plot(np.log10(group_psi.threshold), group_psi["nav"], label="L={0}".format(group.L.iloc[0]), linestyle="-", marker="o", color="C"+str(n), ms=ms)
    
    ax = axs
    # ax.text(0.85, 3, r"B")
    ax.plot(np.log10(group.threshold), group["AbortedMeanOverall"], label="L={0}".format(group.L.iloc[0]), linestyle="-", marker="o", color="C"+str(n), ms=ms)

    x = np.log10(group.threshold)
    y = group["AbortedMeanOverall"]

    z = np.polyfit(x, y, 2)
    f = np.poly1d(z)
    
    x_new = np.linspace(np.log10(2e3), np.log10(upper_lims[group_L]), num=100, endpoint=True)
    ax.plot(x_new, f(x_new), label="_nolegend_", linestyle="--", marker="None", color="C"+str(n), alpha=0.5)
    n+=1
    
# axs[0].set_xlim(2.5,np.log10(2e10))
# axs[0].set_ylim(0,1.05)
# axs[0].set_ylabel(r"Navigability, $\left<\psi\right>$")

ax = axs
ax.set_xlabel(r"Computational threshold, $\log_{10} T$")
ax.set_ylabel(r"Proportion aborted, $\alpha$")
ax.set_xlim(2.5,np.log10(2e10))
ax.set_ylim(0.01,3)
ax.set_yscale('log')

plt.plot([np.log10(3e6), np.log10(3e6)], [-1,2], color="k", zorder=-1, linestyle=":", label="1 month limit", alpha=0.2)
ax.legend(loc="lower right", ncol=3, prop={'size':"x-small"})
ax.legend(loc="upper right", ncol=3, prop={'size':"x-small"})
plt.savefig(os.path.join(OUT_PATH, "fig5_review.pdf"), transparent=True, bbox_inches='tight')